In [ ]:
using Plots
using Random
using LaTeXStrings
using FFTW
using BenchmarkTools

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfont=font(12), lw=2,ms=8)

# Sampling from a Gaussian Random Field
To sample from $N(0,C)$, a mean zero Gaussian random field, we need to specifiy a covariance operator.  Here we will use
$$
C = (-d^2/dx^2)^{-1} + \text{Boundary Conditions}
$$
on the interval $(0,1)$.  Boundary conditions must be specified for this operator to be well-defined.  

# Sample a Gaussian Random Field by Direct Summation with Dirichlet Boundary Conditions
This shows how we can sample a Gaussian random variable by direct summation.  In this excample, the mean is zero, and the covariance is
$$
C = (-d^2/dx^2)^{-1} + \text{Dirichlet Boundary Conditions}
$$
The eigenvalue/eigenfunction pairs are
$$
\lambda_k = \frac{1}{(\pi k)^2}, \quad \varphi_k(x) = \sqrt{2}\sin(\pi k x)
$$

In [ ]:
function sample_field(x, N)
    X = zeros(length(x)); #preallocate storage for the r.v.

    # add all the contributions, term by term from the series
    for k in 1:N
       X .+= sqrt(1/(π*k)^2) * randn() * sqrt(2) * sin.(π*k*x);
    end    
    
    return X
    
end

In [ ]:
nx = 256; # nx + 1 interior mesh points
x = LinRange(0,1, nx+1)[2:end-1]; # interior nodes 

N = 256; # number of terms in the series expansion to include

In [ ]:
Random.seed!(100);
X1 = sample_field(x,N);
X2 = sample_field(x,N);
X3 = sample_field(x,N);

In [ ]:
plot(x, X1,label="")
plot!(x, X2,label="")
plot!(x, X3,label="")
xlabel!(L"$x$")
ylabel!(L"$X$")
title!("Sample Realizations")

In [ ]:
@btime sample_field(x, N);

# Sampling by FFT
Repeat of the above, but with FFT sampler

In [ ]:
function sample_field_fft(N)
    uhat = zeros(ComplexF64, 2 * N) # preallocate space

    ξ = randn(N) # generate the random variables

    # construct the eigenvalues
    k = 1:N
    λ = @. 1 / (π * k)^2

    # fill in the nonzero entries
    # NOTE we need to multiply by 2 * N for FFT scaling
    @. uhat[2:N+1] = 2 * N * sqrt(λ) * sqrt(2) * ξ

    # invert and get the relevant imaginary part
    u = imag.(ifft(uhat))[N+2:end]
    return u
end

In [ ]:
N = 256;
x = LinRange(0, 1, N + 1)[2:end-1];

Random.seed!(100);
X1 = sample_field_fft(N);
X2 = sample_field_fft(N);
X3 = sample_field_fft(N);

In [ ]:
plot(x, X1, label="")
plot!(x, X2, label="")
plot!(x, X3, label="")
xlabel!(L"$x$")
ylabel!(L"$X$")
title!("Sample Realizations")

In [ ]:
@btime sample_field_fft(N);

# Other Boundary Conditions
We can access other boundary conditions with similar tricks.

## Neumann Boundary Conditions

In [ ]:
function sample_field_neumann(N)
    uhat = zeros(ComplexF64, 2 * N) # preallocate space

    ξ = randn(N) # generate the random variables

    # construct the eigenvalues
    k = 1:N
    λ = @. 1 / (π * k)^2

    # fill in the nonzero entries
    # NOTE we need to multiply by 2 *N for FFT scaling
    @. uhat[2:N+1] = 2 * N * sqrt(λ) * sqrt(2) * ξ

    # invert and get the relevant real part
    u_ = real.(ifft(uhat))
    u = [u_[N+1:end]; u_[1]] # get the solution on [-1,1], including endpoints
    return u
end

In [ ]:
N = 512; # set number of spatial points/KLSE modes
x = LinRange(0, 1, N + 1);

Random.seed!(200);
X1 = sample_field_neumann(N);
X2 = sample_field_neumann(N);
X3 = sample_field_neumann(N);

plot(x, X1, label="")
plot!(x, X2, label="")
plot!(x, X3, label="")
xlabel!(L"$x$")
title!("Sample Realizations")

## Periodic Boundary Conditions

In [ ]:
function sample_field_pbc(N)
    uhat = zeros(ComplexF64, N) # preallocate space
    ξ = zeros(ComplexF64, N)

    # set the eigenvalues
    k = [0:N÷2; -N÷2+1:1:-1]
    λ = zeros(N)
    @. λ[2:end] = (1 / (2 * π * k[2:end]))^2
    λ[N÷2+1] = 0 # zero out the assymetric term for k = N÷2

    # generate the random variables 
    @. ξ[2:N÷2] = randn(ComplexF64)
    # ensure complex conjugacy
    ξ[N÷2+2:end] .= conj(ξ[N÷2:-1:2])

    @. uhat = N * sqrt(λ) * sqrt(2) * ξ
    # invert and take real part to remove any floating point
    u = ifft(uhat)
    return real.(u)
    # return u
end

In [ ]:
N = 512; # set number of spatial points/KLSE modes
x = LinRange(0, 1, N + 1)[1:end-1];

Random.seed!(200);
X1 = sample_field_pbc(N);
X2 = sample_field_pbc(N);
X3 = sample_field_pbc(N);


plot(x, X1, label="")
plot!(x, X2, label="")
plot!(x, X3, label="")
xlabel!(L"$x$")
title!("Sample Realizations")
